# 🧠 Rare Disease GraphRAG QA Pipeline
基于 `medical_doc1.pdf` 构建知识图谱 + 图谱增强问答

In [ ]:

!pip install -U langchain openai faiss-cpu networkx matplotlib sentence-transformers pypdf


In [ ]:

import os, re
import networkx as nx
import matplotlib.pyplot as plt
from difflib import get_close_matches
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI


In [ ]:

os.environ['OPENAI_API_KEY'] = 'your-api-key'  # 请替换为你的 OpenAI API 密钥


In [ ]:

loader = PyPDFLoader("medical_doc1.pdf")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
print(f"✅ 文档加载完成，分块数量：{len(chunks)}")


In [ ]:

llm = OpenAI(temperature=0)
all_text = "\n".join([c.page_content for c in chunks[:15]])[:8000]
prompt = """Extract all rare disease names mentioned in the following biomedical text.
Return only as a Python list of strings (no explanation).

Text:
""" + all_text
response = llm.invoke(prompt)
rare_diseases = eval(response)
print(f"🧬 共识别出 {len(rare_diseases)} 种 rare diseases。\n")
print("📋 示例名称:", rare_diseases[:10])


In [ ]:

def normalize(text):
    return re.sub(r"\(.*?\)", "", text.lower()).strip()

def extract_triplets(text):
    pattern = r"\(([^()]+?), ([^()]+?), ([^()]+?)\)"
    return re.findall(pattern, text)

G = nx.DiGraph()
for chunk in chunks:
    prompt = f"""Extract factual triples (head, relation, tail) from this medical text:\n\n{chunk.page_content}\n\nReturn only (head, relation, tail)."""
    result = llm.invoke(prompt)
    for h, r, t in extract_triplets(result):
        G.add_node(normalize(h))
        G.add_node(normalize(t))
        G.add_edge(normalize(h), normalize(t), relation=r.strip())

print(f"✅ 知识图谱构建完成，包含 {len(G.nodes)} 个节点，{len(G.edges)} 条边。")


In [ ]:

plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color="skyblue", edge_color="gray", node_size=1000, font_size=8)
edge_labels = nx.get_edge_attributes(G, "relation")
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=6)
plt.title("Rare Disease Knowledge Graph")
plt.tight_layout()
plt.show()


In [ ]:

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(chunks, embedding_model)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectordb.as_retriever(), return_source_documents=False)


In [ ]:

def graphrag_query(question):
    ent_prompt = f"""Extract the rare disease or symptom entities from this question.\nReturn as a Python list:\n\n{question}"""
    response = llm.invoke(ent_prompt)
    try:
        extracted = eval(response.strip())
    except:
        extracted = []

    hits = []
    for e in extracted:
        match = get_close_matches(normalize(e), G.nodes, n=1, cutoff=0.5)
        if match: hits.append(match[0])

    if not hits:
        print("⚠️ 未在图谱中匹配到实体，改为纯文档问答")
        return qa_chain.invoke(question)["result"]

    neighbor_contexts = []
    for node in hits:
        neighbors = list(nx.single_source_shortest_path_length(G, node, cutoff=2))
        neighbor_contexts.append(f"Entity: {node}\nNeighbors: {', '.join(neighbors)}")

    vector_docs = vectordb.similarity_search(question, k=2)
    vector_text = "\n".join(doc.page_content for doc in vector_docs)
    graph_text = "\n".join(neighbor_contexts)

    final_prompt = f"""Answer based on graph and document knowledge:\n\nGraph:\n{graph_text}\n\nDocs:\n{vector_text}\n\nQuestion: {question}"""
    return llm.invoke(final_prompt)


In [ ]:

query = "What is Ataxia telangiectasia (AT)?"
print("🧠 问题：", query)
print("🤖 回答：", graphrag_query(query))
